In [ ]:
# https://skyvia.com/gallery/inserting-multiple-rows-in-a-single-postgresql-query

In [1]:
import geopandas as gpd
import os
import psycopg2
from shapely.wkt import loads as wkt_loads

from keys import pg_user, pg_pass, pg_host, pg_port, pg_db, pg_table

%matplotlib inline

# census data is in 4269, so project each to 4326 to match footprints
srid = 4326 #ms building footprints srid
crs = {'init':'epsg:4326'} #ms building footprints crs

In [2]:
connection = psycopg2.connect(database=pg_db,
                              user=pg_user,
                              password=pg_pass,
                              host=pg_host,
                              port=pg_port)
connection.set_session(autocommit=True)
cursor = connection.cursor()

## States

In [3]:
gdf = gpd.read_file('data/states')
gdf = gdf.to_crs(crs)
gdf.head(3)

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,02,01785533,0400000US02,02,AK,Alaska,00,1478588231566,277723861311,"(POLYGON ((-173.074642 60.704657, -172.912636 ..."
1,06,01779778,0400000US06,06,CA,California,00,403483182192,20484637928,"(POLYGON ((-118.593969 33.467198, -118.484785 ..."
2,08,01779779,0400000US08,08,CO,Colorado,00,268425964573,1178495763,"POLYGON ((-109.059962 38.499987, -109.05996197..."


In [4]:
# drop states table if it already exists, then create states table
query = """
DROP TABLE IF EXISTS states;
CREATE TABLE states (id SERIAL PRIMARY KEY,
                     geoid VARCHAR NOT NULL,
                     stusps VARCHAR NOT NULL,
                     aland BIGINT NOT NULL);
SELECT AddGeometryColumn ('states', 'geom', %s, 'MULTIPOLYGON', 2);
CREATE INDEX state_index ON states USING GIST(geom);
"""
data = [srid]
cursor.execute(query, data)

In [5]:
%%time
cursor.execute("DELETE FROM states")

# insert each state into the states table one at a time
for label, row in gdf.iterrows():
    geoid = row['GEOID']
    stusps = row['STUSPS']
    aland = row['ALAND']
    geometry_wkt = row['geometry'].wkt
    
    query = """INSERT INTO states (geoid, stusps, aland, geom) 
               VALUES (%s, %s, %s, ST_Multi(ST_GeomFromText(%s, %s)))"""
    data = (geoid, stusps, aland, geometry_wkt, srid)
    cursor.execute(query, data)

Wall time: 85.9 ms


In [6]:
%%time
cursor.execute('VACUUM(FULL, ANALYZE) states;')

Wall time: 134 ms


## Places

In [7]:
%%time
input_folder = 'data/places'
gdf = gpd.GeoDataFrame()
for state in os.listdir(input_folder):
    gdf = gdf.append(gpd.read_file('{}/{}'.format(input_folder, state)))

# outside of hawaii, only retain non-CDPs    
mask = (gdf['STATEFP'] == '15') | (~gdf['NAMELSAD'].str.contains('CDP'))
gdf = gdf[mask]
gdf = gdf.to_crs(crs)

Wall time: 12.9 s


In [8]:
print(len(gdf))
gdf[gdf['NAMELSAD'].str.contains('Tucson')]

19678


,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
65,04,77000,02412104,0477000,Tucson,Tucson city,25,C1,Y,N,G4110,A,613878921,819373,+32.1530356,-110.8707734,"POLYGON ((-111.058731 32.206795, -111.054463 3..."
69,04,68850,02411943,0468850,South Tucson,South Tucson city,25,C1,N,N,G4110,A,2659684,0,+32.1954741,-110.9691542,"POLYGON ((-110.977608 32.203157, -110.977499 3..."


In [9]:
%%time
# drop places table if it already exists, then create places table
query = """
DROP TABLE IF EXISTS places;
CREATE TABLE places (id SERIAL PRIMARY KEY,
                     geoid VARCHAR NOT NULL,
                     name VARCHAR NOT NULL,
                     aland BIGINT NOT NULL);
SELECT AddGeometryColumn ('places', 'geom', %s, 'MULTIPOLYGON', 2);
CREATE INDEX place_index ON places USING GIST(geom);
"""
data = [srid]
cursor.execute(query, data)

Wall time: 79.2 ms


In [10]:
%%time
cursor.execute("DELETE FROM places")

# insert each place into the places table one at a time
for label, row in gdf.iterrows():
    geoid = row['GEOID']
    name = row['NAME']
    aland = row['ALAND']
    geometry_wkt = row['geometry'].wkt
    
    query = """INSERT INTO places (geoid, name, aland, geom) 
               VALUES (%s, %s, %s, ST_Multi(ST_GeomFromText(%s, %s)))"""
    data = (geoid, name, aland, geometry_wkt, srid)
    cursor.execute(query, data)

Wall time: 47.4 s


In [11]:
%%time
cursor.execute('VACUUM(FULL, ANALYZE) places;')

Wall time: 7.67 s


## Urbanized Areas

In [12]:
gdf = gpd.read_file('data/tl_2017_us_uac10/')
gdf = gdf[~(gdf['NAMELSAD10'].str.contains('Urban Cluster'))]
gdf = gdf.to_crs(crs)
len(gdf)

497

In [13]:
gdf[gdf['NAMELSAD10'].str.contains('Tucson')]

,UACE10,GEOID10,NAME10,NAMELSAD10,LSAD10,MTFCC10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
2638,88732,88732,"Tucson, AZ","Tucson, AZ Urbanized Area",75,G3500,U,S,915291544,2086650,+32.2548912,-110.9461410,"(POLYGON ((-111.088915 32.402834, -111.089761 ..."


In [14]:
%%time
# drop urbanized_areas table if it already exists, then create urbanized_areas table
query = """
DROP TABLE IF EXISTS urbanized_areas;
CREATE TABLE urbanized_areas (id SERIAL PRIMARY KEY,
                              geoid VARCHAR NOT NULL,
                              name VARCHAR NOT NULL,
                              aland BIGINT NOT NULL);
SELECT AddGeometryColumn ('urbanized_areas', 'geom', %s, 'MULTIPOLYGON', 2);
CREATE INDEX urbanized_area_index ON urbanized_areas USING GIST(geom);
"""
data = [srid]
cursor.execute(query, data)

Wall time: 36 ms


In [15]:
%%time
cursor.execute("DELETE FROM urbanized_areas")

# insert each urbanized area into the urbanized_areas table one at a time
for label, row in gdf.iterrows():
    geoid = row['GEOID10']
    name = row['NAMELSAD10']
    aland = row['ALAND10']
    geometry_wkt = row['geometry'].wkt
    
    query = """INSERT INTO urbanized_areas (geoid, name, aland, geom) 
               VALUES (%s, %s, %s, ST_Multi(ST_GeomFromText(%s, %s)))"""
    data = (geoid, name, aland, geometry_wkt, srid)
    cursor.execute(query, data)

Wall time: 20.9 s


In [16]:
%%time
cursor.execute('VACUUM(FULL, ANALYZE) urbanized_areas;')

Wall time: 4.54 s


## Test

In [17]:
# all footprints within some city
query = """
        SELECT ST_AsText(footprints.geom)
        FROM footprints, places
        WHERE places.name='Tucson'
        AND ST_Intersects(places.geom, footprints.geom);
        """

In [18]:
%%time
cursor.execute(query)
rows = cursor.fetchall()

Wall time: 8.8 s


In [19]:
%%time
gdf = gpd.GeoDataFrame(rows, columns=['geometry'])
gdf['geometry'] = gdf['geometry'].map(lambda x: wkt_loads(x))
print(len(gdf))

171294
Wall time: 3.69 s


In [20]:
# all footprints within some urbanized area
query = """
        SELECT ST_AsText(footprints.geom)
        FROM footprints, urbanized_areas
        WHERE urbanized_areas.name LIKE '%Tucson%AZ%'
        AND ST_Intersects(urbanized_areas.geom, footprints.geom);
        """

In [21]:
%%time
cursor.execute(query)
rows = cursor.fetchall()

Wall time: 15.8 s


In [22]:
%%time
gdf = gpd.GeoDataFrame(rows, columns=['geometry'])
gdf['geometry'] = gdf['geometry'].map(lambda x: wkt_loads(x))
print(len(gdf))

302322
Wall time: 7.03 s


In [23]:
# all footprints within some state
query = """
        SELECT ST_AsText(footprints.geom)
        FROM footprints, states
        WHERE states.stusps='DE'
        AND ST_Intersects(states.geom, footprints.geom);
        """

In [24]:
%%time
cursor.execute(query)
rows = cursor.fetchall()

Wall time: 6.41 s


In [25]:
%%time
gdf = gpd.GeoDataFrame(rows, columns=['geometry'])
gdf['geometry'] = gdf['geometry'].map(lambda x: wkt_loads(x))
print(len(gdf))

329944
Wall time: 7.98 s


In [26]:
cursor.close()
connection.close()